## 1.1 Install Missed Packages With pip

In [54]:
import sys
import subprocess
import ensurepip

# Ensure pip exists and install required packages if missing
import importlib.util

# 1) Ensure pip is available
try:
    import pip  # noqa: F401
except Exception:
    try:
        ensurepip.bootstrap()
    except Exception:
        # fallback to get-pip.py
        import urllib.request, tempfile, os
        url = "https://bootstrap.pypa.io/get-pip.py"
        fd, tmp_path = tempfile.mkstemp(suffix=".py")
        os.close(fd)
        try:
            urllib.request.urlretrieve(url, tmp_path)
            subprocess.check_call([sys.executable, tmp_path])
        finally:
            try:
                os.remove(tmp_path)
            except Exception:
                pass

# 2) List of packages to ensure (pip-name -> import-name)
required = {
    "pandas": "pandas",
    "numpy": "numpy",
    "matplotlib": "matplotlib",
    "seaborn": "seaborn",
    "scikit-learn": "sklearn",
    "scipy": "scipy",
}

# 3) Detect missing modules
missing = [pkg for pkg, mod in required.items() if importlib.util.find_spec(mod) is None]

# 4) Install missing packages in one pip call
if missing:
    print("Installing missing packages:", missing)
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade"] + missing)

# 5) Report installed versions (or failures)
for pkg, mod in required.items():
    try:
        module = __import__(mod)
        version = getattr(module, "__version__", "unknown")
        print(f"{pkg} ({mod}) version: {version}")
    except Exception:
        print(f"Failed to import {mod}")

pandas (pandas) version: 2.3.3
numpy (numpy) version: 1.26.4
matplotlib (matplotlib) version: 3.6.3
seaborn (seaborn) version: 0.13.2
scikit-learn (sklearn) version: 1.7.2
scipy (scipy) version: 1.11.4


In [55]:
# Retry installation with --break-system-packages flag if needed
if missing:
    print("Retrying installation with --break-system-packages...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "--break-system-packages"] + missing)

## 1.2 Import Necessary libraries

In [56]:
# Importing the relevant libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

from scipy import stats

## 1.3 Helper Functions

In [57]:
def mount_google_drive():
    """
    Checks if running in Google Colab and mounts Google Drive if so.
    Returns the base path to the user's Drive content if successful, otherwise None.
    """
    try:
        from google.colab import drive
        print("Google Colab environment detected. Mounting Google Drive...")
        drive.mount('/content/drive')
        # Return the common base path for Google Drive
        return '/content/drive/MyDrive'
    except ImportError:
        # This will be the case on your local machine
        print("Not running in a Google Colab environment. Skipping Google Drive mount.")
        return None
    except Exception as e:
        print(f"An error occurred while mounting Google Drive: {e}")
        return None

## 2 Loading the dataset

In [58]:
# List to store DataFrames
dfs = []

# --- Define Paths ---
local_data_path = './data'  # The local directory to check first
# IMPORTANT: Update this path to your dataset's location in Google Drive
google_drive_folder = 'CIC-IDS-2017'  # The specific folder name in your MyDrive

def load_data_from_path(path, df_list):
    """Walks a directory and loads all found .csv files into a list of DataFrames."""
    if not (os.path.exists(path) and os.path.isdir(path)):
        print(f"Data directory not found at: {path}")
        return
    
    print(f"Searching for .csv files in: {path}")
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('.csv'):
                file_path = os.path.join(dirname, filename)
                print(f"  - Reading file: {file_path}")
                try:
                    df_list.append(pd.read_csv(file_path))
                except Exception as e:
                    print(f"    - Error reading {file_path}: {e}")

# --- 1. Attempt to load from local path ---
load_data_from_path(local_data_path, dfs)

# --- 2. If no local data, attempt to load from Google Drive ---
if not dfs:
    print("\nNo local data loaded. Attempting to use Google Drive.")
    drive_base_path = mount_google_drive()
    if drive_base_path:
        # Construct the full path to your data on Google Drive
        gdrive_data_path = os.path.join(drive_base_path, google_drive_folder)
        load_data_from_path(gdrive_data_path, dfs)

# --- 3. Final check ---
if not dfs:
    print("\nWarning: No data could be loaded. The 'dfs' list is empty.")
else:
    print(f"\nSuccessfully loaded {len(dfs)} DataFrame(s).")

Searching for .csv files in: ./data
  - Reading file: ./data/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
  - Reading file: ./data/Friday-WorkingHours-Morning.pcap_ISCX.csv
  - Reading file: ./data/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
  - Reading file: ./data/Wednesday-workingHours.pcap_ISCX.csv
  - Reading file: ./data/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
  - Reading file: ./data/Monday-WorkingHours.pcap_ISCX.csv
  - Reading file: ./data/Tuesday-WorkingHours.pcap_ISCX.csv
  - Reading file: ./data/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv

Successfully loaded 8 DataFrame(s).


In [59]:
# Data dimensions of each individual dataset
for i, data in enumerate(dfs, start=1):
    rows, cols = data.shape
    print(f'df{i} -> {rows} rows, {cols} columns')

df1 -> 288602 rows, 79 columns
df2 -> 191033 rows, 79 columns
df3 -> 170366 rows, 79 columns
df4 -> 692703 rows, 79 columns
df5 -> 225745 rows, 79 columns
df6 -> 529918 rows, 79 columns
df7 -> 445909 rows, 79 columns
df8 -> 286467 rows, 79 columns


## 1.2. Merging the DataFrames

In [60]:
# It's good practice to check if the list is populated before concatenation.
if dfs:
    # Concatenate all DataFrames into a single DataFrame
    data = pd.concat(dfs, axis=0, ignore_index=True)
    print("DataFrames concatenated successfully.")

    # Deleting the list of DataFrames to free up memory
    del dfs
else:
    # Handle the case where no DataFrames were created
    print("Warning: No data was found to concatenate. The 'data' DataFrame has not be created.")
    data = pd.DataFrame() # Optionally, create an empty DataFrame


DataFrames concatenated successfully.


In [61]:
# Display the first few rows
data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,166,1,1,0,0,0,0,0.0,0.0,...,32,0.000,0.000,0,0,0.0,0.000,0,0,BENIGN
1,60148,83,1,2,0,0,0,0,0.0,0.0,...,32,0.000,0.000,0,0,0.0,0.000,0,0,BENIGN
2,123,99947,1,1,48,48,48,48,48.0,0.0,...,40,0.000,0.000,0,0,0.0,0.000,0,0,BENIGN
3,123,37017,1,1,48,48,48,48,48.0,0.0,...,32,0.000,0.000,0,0,0.0,0.000,0,0,BENIGN
4,0,111161336,147,0,0,0,0,0,0.0,0.0,...,0,1753752.625,2123197.578,4822992,95,9463032.7,2657727.996,13600000,5700287,BENIGN


In [62]:
# Display 10 random rows
data.sample(n=10, random_state=42)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
746827,80,217872,3,7,382,11595,382,0,127.333333,220.547803,...,32,0.0,0.0,0,0,0.0,0.0,0,0,DoS Hulk
946912,80,84743121,5,7,355,11595,343,0,71.000000,152.082215,...,20,11013.0,0.0,11013,11013,84600000.0,0.0,84600000,84600000,DoS Hulk
2216843,53,2714807,6,4,258,416,45,42,43.000000,1.549193,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
699389,443,11053475,21,22,18983,8109,2389,0,903.952381,1125.348945,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1170268,80,15220,1,1,6,6,6,6,6.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
800686,80,99394898,8,6,339,11595,333,0,42.375000,117.449001,...,20,1945.0,0.0,1945,1945,99200000.0,0.0,99200000,99200000,DoS Hulk
1434488,80,109263,3,4,26,11601,20,0,8.666667,10.263203,...,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
1968368,53,69709,4,4,168,768,42,42,42.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
934343,80,83027396,10,6,311,11595,311,0,31.100000,98.346835,...,32,1984.0,0.0,1984,1984,82900000.0,0.0,82900000,82900000,DoS Hulk
693547,80,61462028,5,5,231,4559,213,0,46.200000,93.280223,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [63]:
# Get dataset dimensions
print(f"Dataset Dimensions: {data.shape}")

Dataset Dimensions: (2830743, 79)


In [64]:
# Display data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

By checking the data types, it is safe to assume the dataset countains only metric features, considering 'Label' as 'y' (the only categorical column).

In [65]:
# Checking for missing values
missing_values = data.isna().sum()
missing_percentage = (missing_values / len(data)) * 100

# Printing columns with missing values
for column, count in missing_values.items():
    if count != 0:
        print(f"Column '{column}' has {count} missing values, which is {missing_percentage[column]:.2f}% of the total")

Column 'Flow Bytes/s' has 1358 missing values, which is 0.05% of the total


In [66]:
# Checking and counting duplicates
duplicates = data.duplicated()
duplicate_count = duplicates.sum()

# Output results
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 308381


## 3. Data Cleaning

In [67]:
# Removal of leading/trailing whitespace
col_names = {col: col.strip() for col in data.columns}
data.rename(columns = col_names, inplace = True)

# 3.1. Duplicates and Infinite

In [68]:
# Checking and counting duplicates
duplicates = data.duplicated()
duplicate_count = duplicates.sum()

# Output results
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 308381


Removing duplicates from a dataset is often considered safe and beneficial as it enhances data integrity, improves statistical accuracy, increases efficiency, and simplifies analysis.

In [69]:
# Removal of duplicates
data = data.drop_duplicates(keep='first')
del duplicates
data.shape

(2522362, 79)

The same can be done for columns

In [70]:
# Identify columns with identical data
identical_columns = {}
columns = data.columns
list_control = columns.copy().tolist()

# Compare each pair of columns
for col1 in columns:
    for col2 in columns:
        if col1 != col2:
            if data[col1].equals(data[col2]):
                if (col1 not in identical_columns) and (col1 in list_control):
                    identical_columns[col1] = [col2]
                    list_control.remove(col2)
                elif (col1 in identical_columns) and (col1 in list_control):
                    identical_columns[col1].append(col2)
                    list_control.remove(col2)

# Print the result
if identical_columns:
    print("Identical columns found:")
    for key, value in identical_columns.items():
        print(f"'{key}' is identical to {value}")
else: print("No identical columns found.")

Identical columns found:
'Total Fwd Packets' is identical to ['Subflow Fwd Packets']
'Total Backward Packets' is identical to ['Subflow Bwd Packets']
'Fwd PSH Flags' is identical to ['SYN Flag Count']
'Bwd PSH Flags' is identical to ['Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']
'Fwd URG Flags' is identical to ['CWE Flag Count']
'Fwd Header Length' is identical to ['Fwd Header Length.1']


In [71]:
# Removing the columns with duplicated values
for key, value in identical_columns.items():
    data.drop(columns=value, inplace=True)

In [72]:
print(data.columns)
data.shape

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'FIN Flag Count', 'RST Flag Count',
       'PSH Flag C

(2522362, 67)

In [73]:
# Checking for infinite values
num_columns = data.select_dtypes(include = np.number).columns
has_infinite = np.isinf(data[num_columns]).sum()
print(has_infinite[has_infinite > 0])

Flow Bytes/s      1211
Flow Packets/s    1564
dtype: int64


Removing infinite values is typically safe and beneficial, as it enhances data integrity, ensures statistical accuracy, aids in proper model training, and clarifies insights.

In [74]:
# Treating infinite values
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# 3.1 Missing Values

There are different approaches to dealing with missing values. The first step in identifying how to proceed is to understand their impact on the dataset. Here, we will do that by analyzing the y column (Label).

In [75]:
# Attack counts
attack_counts = data['Label'].value_counts().reset_index()
attack_counts.columns = ['Attack Type', 'Number of Occurrences']

# Duplicating the df and dropping rows with missing values
data_no_na = data.dropna()

# Counting the total number of occurrences of each attack after dropping
occurrences_nonull = data_no_na['Label'].value_counts().reset_index()
occurrences_nonull.columns = ['Attack Type', 'Occurrences w/o Null Rows']

# Merging the DataFrames
attack_counts = attack_counts.merge(occurrences_nonull, on='Attack Type', how='left')

# Calculating the difference
attack_counts['Abs Difference'] = attack_counts['Number of Occurrences'] - attack_counts['Occurrences w/o Null Rows']
attack_counts['Difference %'] = ((attack_counts['Abs Difference'] * 100) / attack_counts['Number of Occurrences']).round(2)

# Visualization
attack_counts

,Attack Type,Number of Occurrences,Occurrences w/o Null Rows,Abs Difference,Difference %
0,BENIGN,2096484,2095057,1427,0.07
1,DoS Hulk,172849,172846,3,0.00
2,DDoS,128016,128014,2,0.00
3,PortScan,90819,90694,125,0.14
4,DoS GoldenEye,10286,10286,0,0.00
5,FTP-Patator,5933,5931,2,0.03
6,DoS slowloris,5385,5385,0,0.00
7,DoS Slowhttptest,5228,5228,0,0.00
8,SSH-Patator,3219,3219,0,0.00
9,Bot,1953,1948,5,0.26


In [76]:
# Cleaning up
del data_no_na

# Evaluating percentage of missing values per column
threshold = 10
missing_percentage = (data.isnull().sum() / len(data)) * 100

# Filter columns with missing values over the threshold
high_missing_cols = missing_percentage[missing_percentage > threshold]

# Print columns with high missing percentages
if len(high_missing_cols) > 0:
    print(f'The following columns have over {threshold}% of missing values:')
    print(high_missing_cols)
else:
    print('There are no columns with missing values greater than the threshold')

There are no columns with missing values greater than the threshold


The analysis of missing values across the dataset suggests that missing values are not heavily concentrated in any single column and that the dataset can tolerate row-wise removal without significant impact.

In [77]:
row_missing_percentage = (data.isna().sum(axis=1) / data.shape[1]) * 100
print(row_missing_percentage.describe())

missing_rows = data.isna().any(axis=1).sum()
print(f'\nTotal rows with missing values: {missing_rows}')

count    2.522362e+06
mean     1.850907e-03
std      7.430795e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.985075e+00
dtype: float64

Total rows with missing values: 1564


Following previous studies, such as Panigrahi and Borah (2018), and considering the minimal impact of removing rows with missing values (concentrated on the Benign class), we will proceed to drop these rows from the DataFrame.

In [78]:
# Dropping missing values
data = data.dropna()
print(f'Dataset shape after row-wise removal: {data.shape}')

Dataset shape after row-wise removal: (2520798, 67)


# 3.2 Data-Driven Feature Selection

The idea here is to perform a straightforward manual feature selection process that focuses on examining the dataset critically.

In [80]:
# Check for numeric columns that contain only a single unique value, indicating no diversity in values.
# Such columns contribute no useful information to the analysis and may be candidates for removal.
only_unique_cols = []
for col in data.columns:
    if len(data[col].unique()) == 1:
        only_unique_cols.append(col)
        print(col)

print(f'\nThe number of columns with only one unique values is: {len(only_unique_cols)}')

Bwd PSH Flags

The number of columns with only one unique values is: 1


Columns with only a single unique value can typically be removed from the dataset before training machine learning models without risk of loss of relevant information.

In [81]:
# Dropping the columns with only one unique value
data.drop(only_unique_cols, axis=1, inplace=True)
del only_unique_cols

In [82]:
# Checking the new shape after cleaning
data.shape

(2520798, 66)

## 3.3. Grouping/ Removing Attack Types

As seen previously, the dataset categorizes various types of attacks into distinct groups, including different variations of Denial of Service (DoS) attacks. Given the imbalance present in the dataset, consolidating these classes could enhance the training process of ML models. This approach allows models to learn from a more balanced representation of the data, potentially improving classification accuracy and model performance.

### 3.3.1. Grouping attack types

The table below illustrates the proposed grouping of attack categories. For now, DoS and DDoS will remain separate due to their distinct characteristics, such as differences in attack methodology and mitigation strategies, which could impact the model's performance and interpretation.


| **Group**                     | **Attack Type**             | **Count**  |  
|-------------------------------|-----------------------------|------------|  
| **Normal Traffic**            | BENIGN                      | 2,095,057  |  
|                               |                             | **Total: 2,095,057** |  
| **DoS Attacks**              | DoS Hulk                   | 172,846    |  
|                               | DoS GoldenEye               | 10,286     |  
|                               | DoS Slowloris               | 5,385      |  
|                               | DoS Slowhttptest            | 5,228      |
|                               |                             | **Total: 193,745** |
| **DDoS Attacks**              | DDoS                        | 128,014     |
|                               |                             | **Total: 128,014** |
| **Port Scanning**            | PortScan                    | 90,694     |  
|                               |                             | **Total: 90,694**  |  
| **Brute Force Attacks**      | FTP-Patator                 | 5,931      |  
|                               | SSH-Patator                 | 3,219      |  
|                               |                             | **Total: 9,150**   |  
| **Bots**                      | Bot                         | 1,948      |  
|                               |                             | **Total: 1,948**   |  
| **Web Attacks**              | Web Attack – Brute Force    | 1,470      |  
|                               | Web Attack – XSS            | 652        |  
|                               | Web Attack – SQL Injection   | 21         |  
|                               |                             | **Total: 2,143**   |  
| **Infiltration Attacks**     | Infiltration                | 36         |  
|                               |                             | **Total: 36**      |  
| **Miscellaneous**            | Heartbleed                  | 11         |  
|                               |                             | **Total: 11**      |

# Mapping the attacks to the new group
group_mapping = {
    'BENIGN': 'Normal Traffic',
    'DoS Hulk': 'DoS',
    'DDoS': 'DDoS',
    'PortScan': 'Port Scanning',
    'DoS GoldenEye': 'DoS',
    'FTP-Patator': 'Brute Force',
    'DoS slowloris': 'DoS',
    'DoS Slowhttptest': 'DoS',
    'SSH-Patator': 'Brute Force',
    'Bot': 'Bots',
    'Web Attack � Brute Force': 'Web Attacks',
    'Web Attack � XSS': 'Web Attacks',
    'Infiltration': 'Infiltration',
    'Web Attack � Sql Injection': 'Web Attacks',
    'Heartbleed': 'Miscellaneous'
}

# Map to new group column
data['Attack Type'] = data['Label'].map(group_mapping)

In [84]:
# Mapping the attacks to the new group
group_mapping = {
    'BENIGN': 'Normal Traffic',
    'DoS Hulk': 'DoS',
    'DDoS': 'DDoS',
    'PortScan': 'Port Scanning',
    'DoS GoldenEye': 'DoS',
    'FTP-Patator': 'Brute Force',
    'DoS slowloris': 'DoS',
    'DoS Slowhttptest': 'DoS',
    'SSH-Patator': 'Brute Force',
    'Bot': 'Bots',
    'Web Attack � Brute Force': 'Web Attacks',
    'Web Attack � XSS': 'Web Attacks',
    'Infiltration': 'Infiltration',
    'Web Attack � Sql Injection': 'Web Attacks',
    'Heartbleed': 'Miscellaneous'
}

# Map to new group column
data['Attack Type'] = data['Label'].map(group_mapping)

In [85]:
# Dropping the old 'Label' column
data.drop(columns='Label', inplace=True)

### 3.3.2. Removal of Attack Types

Given the very low instance counts for 'Infiltration' (36) and 'Heartbleed' (11), it is advisable to remove these categories from the dataset as they can lead to overfitting and unreliable model performance, even techniques like SMOTE may not be sufficient to create a representative training sample.

In [87]:
# Removing rows with statistically irrelevant attack types
data.drop(data[(data['Attack Type'] == 'Infiltration') | (data['Attack Type'] == 'Miscellaneous')].index, inplace=True)

In [88]:
# Data shape and attack counts after removal
print(data.shape)
data['Attack Type'].value_counts()

(2520751, 66)


Attack Type
Normal Traffic    2095057
DoS                193745
DDoS               128014
Port Scanning       90694
Brute Force          9150
Web Attacks          2143
Bots                 1948
Name: count, dtype: int64

In [89]:
# Removing rows with statistically irrelevant attack types
data.drop(data[(data['Attack Type'] == 'Infiltration') | (data['Attack Type'] == 'Miscellaneous')].index, inplace=True)

In [90]:
# Data shape and attack counts after removal
print(data.shape)
data['Attack Type'].value_counts()

(2520751, 66)


Attack Type
Normal Traffic    2095057
DoS                193745
DDoS               128014
Port Scanning       90694
Brute Force          9150
Web Attacks          2143
Bots                 1948
Name: count, dtype: int64

In [ ]:
data.sample(10)